<a href="https://colab.research.google.com/github/boywithdragons/Classify-an-image-using-Gemini/blob/main/Explained%20code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title #__Upgrading the Generative Ai liberary to its latest model.__
!pip install google-generativeai --upgrade

# __Configuring the Gemini API key__

In [ ]:
#@title Importing the Google Gemini AI Software Development Kit __(google.generativeai)__ and assigning it an alias __genai__

import google.generativeai as genai

In [ ]:
#@title Importing the <b>userdate</b> from __google.colob__, which is used for handing user specifi data, Such as storing and retrieving the API keys securely in colab.
from google.colab import userdata

In [ ]:
#@title Importing __types__ from __google.genereativeai__ which will handles __data types__ for the API.
from google.generativeai import types

In [ ]:
#@title Defining the API key name __GOOGLE_API_KEY__ which is being stored in the variable.
#@markdown ######API keys are stored securely. This string __(GOOGLE_API_KEY)__ acts as a reference name to retrieve the actual API key from the source.
GEMINI_API_NAME = "GOOGLE_API_KEY" #@param{type: "string"}
# @markdown __#@param{type: "string"}__ is another Colob-formatted comment used to create an interatve input field in the notebook. Which is as shown above.
# @markdown ###### __type: "string"__ is used so that string value can be used for understanding


In [ ]:
#@title Retrieving and configure the API key.
#@markdown ##### __userdata.get(GEMINI_API_NAME)__ fetches the __API key__ stored in __Google colab secrects__
#@markdown ##### __genai.configure(api_key=GOOGLE_API_KEY)__ passes the key to the __Google Gemini API__, allowing the script to make requests.
#@markdown ##### API __(Generative Language API key)__ has been taken from https://console.cloud.google.com/apis/credentials?project=gen-lang-client-0663598904 and stored in secrets in the colab.

try:
  GOOGLE_API_KEY = userdata.get(GEMINI_API_NAME)
  genai.configure(api_key=GOOGLE_API_KEY)


In [ ]:
#@title Handling errors if API key is missing or invalid.
#@markdown ###### If API key does not exist, prints instruction on hwo to get an API key.
#@markdown ###### Ask the user to store the API key properly in Google Colab
#@markdown ##### Raising an exception using __raise e__.


except userdata.SecretNotFoundError as e:
  print(f"Secert not fount\n\nThis expects you to create a secret named {GEMINI_API_NAME} in Colab\n\nVisit https://makesuite.google.com/app/apikey to create and API key\n\nStore that is the secrets section on the left side of the notebook (key icon)\n\nName the secret{GEMINI_API_NAME}")
  raise e

In [ ]:
#@title Handling access permission issues.
#@markdown ##### Code for showing error if colab does not have permission to access stored secrets to get the API.
#@markdown ##### If this will happens user need to give the access manully by going to secrets > Add new secret.


except userdata.NotebookAccessError as e:
  print(f"You need to grant this notebook access to the {GEMINI_API_NAME} secret in order for the notebook to acces Gemini on your behalf.")


In [ ]:
#@title Handling any other Unknown errors.
#@markdown ##### Error message will suggest to chcek and validate the stored API key and if needed need to re-store the key.
#@markdown ##### Errors might be an APU key is incorrectly formatted, network issue or any unexpected problem.

except Exception as e:
  #unknown error
  print(f"There was an uknow error. Ensure you have a secret {GEMINI_API_NAME} stored in Colab and it's a vaild key from https://makersuite.google.com/app/apikey")
  raise e


In [ ]:
#@title Initializeing the Gemini Model
#@markdown ###### Loaded __gemini-pro-vision__ whichs supports text generation, image and text input processing.
#@markdown ###### Now the model has been ready to process the requests.
model = genai.GenerativeModel("gemini-pro-vision")

# __Demo image analysis__



In [ ]:
#@title Improting required Libraries.
#@markdown ###### __PIL.Image__ is used for handling images.
#@markdown ###### __base64__ encodes the image in Base64 formate for embedding in HTML.
#@markdown ###### __io__ helps handling image data in memory
import PIL.Image, base64, io
#@markdown ###### __IPython.display, HTML, display__ used to render the image and text description in an HTML format.
from IPython.display import HTML, display

In [ ]:
#@title Download the Image form a URL
#@markdown ######__!curl__ runs a shell command inside the notebook. It also fetches image from the internet dynamically insted of using local files.
#@markdown ######__-o image.jpg__ saves the downloaded file as __"image.jpg"__

!curl -o image.jpg https://i.pinimg.com/originals/c5/8d/6d/c58d6d66c80ab56abbbbef8cdcd382f5.jpg

In [ ]:
#@title Define Image Path
#@markdown Stores the image filename in a variable __(imag_path)__
img_path = "image.jpg"

In [ ]:
#@title Opening the Image path
#@markdown It opens the downloaded image using __Pillow(PIL)__ and handles the further processing or manipulation.
img =PIL.Image.open(img_path)

In [ ]:
#@title Uploading Image to Google Gemini AI
image_part = genai.upload_file(img_path)
#@markdown __genai.upload_file(img_path)__ converts the image into a format that Gemini can process.

In [ ]:
#@title Initializeing gemini AI Model.
#@markdown ##### __GenerativeModel__ is used to load the __gemini-1.5-flash__ model.
model = genai.GenerativeModel("gemini-1.5-flash")

In [ ]:
#@title Generate Description of the Image.
#@markdown Sends the uploaded image to Gemini with the prompt __"Describe the Image"__.
response = model.generate_content(
    [image_part, ("Describe the Image.")])

In [ ]:
#@title Convert the image to Base64 for embedding in HTML
buffered = io.BytesIO()
#@markdown ######__buffered = io.BytesIO()__ creates an in-memory temporary storage(buffer).
img.save(buffered, format="JPEG")
#@markdown ######__img.save(buffered, format="JPEG")__ saves the image in the buffer without writing to disk.
img_str = base64.b64encode(buffered.getvalue()).decode()
#@markdown ###### __Base64__ embeds images directly in HTML without requiring an external file link.
#@markdown ###### __base64.b64encode().decode()__ converts the binary data to Base64 format
#@markdown ###### __buffered.getvalue()__ extracts the image as binary data.

In [ ]:
#@title Constuct a table to render the view
#@markdown Created a HTML table to display. __The image__ and __The AI-generated description__

table_html = f"""
<table>
<tr>
    <td><img src= 'data.image/jpeg;based64,{img_str}' style='width:500px; height:auto;'></td>
</tr>
<tr>
    <td style='vertical-align: bottom;'>{response.text}</td>
</tr>
</table>
"""

In [ ]:
#@title Display the table
#@markdown It coverts the string to an __HTML object__
display(HTML(table_html))

# __Uploading your own image.__

In [ ]:
#@markdown ##### __google.colab__ is a module specifically for Google colab.
#@markdown ##### __files__ provides methods for handling file uploads and downloads
from google.colab import files
#@markdown ##### __uploaded = files.upload()__ opens a file selection dialog in Google colab. The selecte files are uploaded to the colab environment an stored in a dictionary
uploaded = files.upload()

#__Asking Gemini about the Image__

In [ ]:
#@title Now ask gemini about your image

img = PIL.Image.open(list(uploaded.keys())[0])

image_part = genai.upload_file(list(uploaded.keys())[0])
# Upload the image file to Gemini

response = model.generate_content([image_part, "Describe the image" ])

#convert the image to Base64 for embedding in HTML
buffered = io.BytesIO()
img.save(buffered, format="JPEG")
img_str = base64.b64encode(buffered.getvalue()).decode()

#construct a table
table_html = f"""
<table>
<tr>
    <td><img scr = 'date:image/jpeg;base64,{img_str}' style='width 500px; height:auto:'></td>
</tr>
<tr>
    <td style = 'vertical-align: bottom;'>{response.text}</td>
</tr>
</table>
"""

# display the table
display(HTML(table_html))
